In [30]:
import findspark
findspark.init()
import pyspark
import pyspark.sql
from pyspark import SparkConf
from pyspark import SparkContext
conf = SparkConf()#
conf.set("spark.python.profile", "true")
#conf = sc.getConf()
conf.set( "spark.driver.memory", "4g" )
conf.set( "spark.jars","/Users/crisliu/agile/lib/mongo-hadoop-spark-2.0.2.jar,\
/Users/crisliu/agile/lib/mongo-java-driver-3.4.0.jar,\
/Users/crisliu/agile/lib/elasticsearch-spark-20_2.10-5.0.0-alpha5.jar,\
/Users/crisliu/agile/lib/snappy-java-1.1.2.6.jar,\
/Users/crisliu/agile/lib/lzo-hadoop-1.0.5.jar")
#/Users/crisliu/agile/lib/elasticsearch-hadoop-5.0.0-alpha5.jar"
#conf.set("spark.executor.extraClassPath","~")
sc = SparkContext('local', 'test', conf=conf)
#sc = pyspark.SparkContext()
APP_NAME = "my_script.py"
spark = pyspark.sql.SparkSession(sc).builder.appName(APP_NAME).getOrCreate()

# get_flight_distance

In [4]:
import sys, os, re
import json

# Load the on-time parquet file
on_time_dataframe = spark.read.parquet('../data/on_time_performance.parquet')
on_time_dataframe.registerTempTable("on_time_performance")

origin_dest_distances = spark.sql("""
  SELECT Origin, Dest, AVG(Distance) AS Distance
  FROM on_time_performance
  GROUP BY Origin, Dest
  ORDER BY Distance
  """)
origin_dest_distances.repartition(1).write.mode("overwrite").json("../data/origin_dest_distances.json")
os.system("cp ../data/origin_dest_distances.json/part* ../data/origin_dest_distances.jsonl")


0

In [5]:
!mongoimport -d agile_data_science -c origin_dest_distances --file \ ../data/origin_dest_distances.jsonl

2017-10-18T22:06:47.714-0400	Failed: open  ../data/origin_dest_distances.jsonl: no such file or directory
2017-10-18T22:06:47.714-0400	imported 0 documents


# Testing Our API

In [11]:
!curl -XPOST 'http://localhost:5000/flights/delays/predict/regress' \
-F 'DepDelay=5.0' -F 'Carrier=AA' -F 'FlightDate=2015-12-23' \
-F 'Dest=ATL' -F 'FlightNum=1519' -F 'Origin=SFO' 

{"Delay": -38.364848600785905}

In [2]:
import iso8601
iso8601.parse_date('2015-12-23')
from web import predict_utils
predict_utils.get_regression_date_args('2015-12-23')

datetime.datetime(2015, 12, 23, 0, 0, tzinfo=datetime.timezone.utc)

In [21]:
!curl -XPOST 'http://localhost:5000/flights/delays/predict/classify' \
-F 'DepDelay=5.0' -F 'Carrier=AA' -F 'FlightDate=2016-12-23' \
-F 'Dest=ATL' -F 'FlightNum=1519' -F 'Origin=SFO' \
| json_pp

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   924  100   275  100   649    404    953 --:--:-- --:--:-- --:--:--  1014
{
   "_id" : {
      "$oid" : "59ffb8584951c415293d3c1a"
   },
   "DayOfWeek" : 4,
   "Carrier" : "AA",
   "DayOfYear" : 358,
   "Timestamp" : "2017-11-05T20:18:16.707646",
   "Origin" : "SFO",
   "DayOfMonth" : 23,
   "DepDelay" : 5,
   "FlightNum" : "1519",
   "Dest" : "ATL",
   "FlightDate" : "2016-12-23",
   "Distance" : 2139
}


# fetch_prediction_requests.py

In [50]:
import json
import datetime, iso8601
import os

# Save to Mongo
from bson import json_util
import pymongo_spark
pymongo_spark.activate()

In [83]:
base_path = '/Users/crisliu/agile/'
iso_date = '2017-11-05'

# Get today and tomorrow's dates as iso strings to scope query
today_dt = iso8601.parse_date(iso_date)
rounded_today = today_dt.date()
iso_today = rounded_today.isoformat()
rounded_tomorrow_dt = rounded_today + datetime.timedelta(days=1)
iso_tomorrow = rounded_tomorrow_dt.isoformat()

# Create mongo query string for today's data
mongo_query_string = """{{
"Timestamp": {{
  "$gte": "{iso_today}",
  "$lte": "{iso_tomorrow}"
}}
}}""".format(
iso_today=iso_today,
iso_tomorrow=iso_tomorrow
)
mongo_query_string = mongo_query_string.replace('\n', '')

In [84]:
# Create the config object with the query string
mongo_query_config = dict()
mongo_query_config["mongo.input.query"] = mongo_query_string

# Load the day's requests using pymongo_spark
prediction_requests = sc.mongoRDD(
        'mongodb://localhost:27017/agile_data_science.prediction_tasks',
        config=mongo_query_config
)

# Build the day's output path: a date based primary key directory structure
today_output_path = "{}Agile_Data_Code_2/data/prediction_tasks_daily.json/{}".format(
base_path,
iso_today
)

# Generate json records
prediction_requests_json = prediction_requests.map(json_util.dumps)

# Write/replace today's output path
os.system("rm -rf {}".format(today_output_path))
prediction_requests_json.saveAsTextFile(today_output_path)
os.system("cp {}/part* {}.jsonl".format(today_output_path,today_output_path))


In [87]:
!cat ../data/prediction_tasks_daily.json/2017-11-05.jsonl

{"Carrier": "AA", "DayOfMonth": 23, "Timestamp": "2017-11-05T20:18:16.707646", "_id": {"$oid": "59ffb8584951c415293d3c1a"}, "Distance": 2139.0, "DayOfYear": 358, "Dest": "ATL", "FlightNum": "1519", "FlightDate": "2016-12-23", "DayOfWeek": 4, "Origin": "SFO", "DepDelay": 5.0}
{"Carrier": "AA", "DayOfMonth": 31, "Timestamp": "2017-11-05T20:21:30.225769", "_id": {"$oid": "59ffb91a4951c415293d3c1b"}, "Distance": 2586.0, "DayOfYear": 366, "Dest": "SFO", "FlightNum": "712", "FlightDate": "2016-12-31", "DayOfWeek": 5, "Origin": "JFK", "DepDelay": 15.0}
{"Carrier": "DL", "DayOfMonth": 31, "Timestamp": "2017-11-05T20:22:09.632064", "_id": {"$oid": "59ffb9414951c415293d3c1c"}, "Distance": 2182.0, "DayOfYear": 366, "Dest": "SEA", "FlightNum": "1519", "FlightDate": "2016-12-31", "DayOfWeek": 5, "Origin": "ATL", "DepDelay": 10.0}


# spark-submit ../ch08/make_predictions.py 2017-11-05 .

In [102]:
! spark-submit ../ch08/make_predictions.py 2017-11-05 /Users/crisliu/agile

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
+-------+----------+---------+---------+--------+----+--------+----------+---------+------+--------------------+
|Carrier|DayOfMonth|DayOfWeek|DayOfYear|DepDelay|Dest|Distance|FlightDate|FlightNum|Origin|           Timestamp|
+-------+----------+---------+---------+--------+----+--------+----------+---------+------+--------------------+
|     AA|        23|        4|      358|     5.0| ATL|  2139.0|2016-12-23|     1519|   SFO|2017-11-05 20:18:...|
|     AA|        31|        5|      366|    15.0| SFO|  2586.0|2016-12-31|      712|   JFK|2017-11-05 20:21:...|
|     DL|        31|        5|      366|    10.0| SEA|  2182.0|2016-12-31|     1519|   ATL|2017-11-05 20:22:...|
+-------+----------+---------+---------+--------+----+--------+----------+---------+------+--------------------+

# Storing Predictions in MongoDB

In [109]:
input_path = base_path + 'data/prediction_results_daily.json/' + '2017-11-05'

In [110]:
# Load and JSONize text
prediction_results_raw = sc.textFile(input_path)
prediction_results = prediction_results_raw.map(json_util.loads)
# Store to MongoDB
prediction_results.saveToMongoDB(
"mongodb://localhost:27017/agile_data_science.prediction_results"
)

In [ ]:
#inspect our results in the Mongo console:
db.prediction_results.find().pretty()

# Setting up Air flow

In [ ]:
 python ~/airflow/dags/setup.py

In [ ]:
airflow list_tasks agile_data_science_batch_prediction_model_training
airflow list_tasks agile_data_science_batch_predictions_daily
airflow test agile_data_science_batch_prediction_model_training pyspark_extract_features 2017-11-05
airflow test agile_data_science_batch_predictions_daily pyspark_fetch_prediction_requests

# Kafka

In [ ]:
kafka/bin/zookeeper-server-start.sh kafka/config/zookeeper.properties

In [ ]:
kafka/bin/kafka-server-start.sh kafka/config/server.properties

In [ ]:
kafka/bin/kafka-topics.sh --create --zookeeper localhost:2181 --replication-factor 1 --partitions 1 --topic flight_delay_classification_request

In [ ]:
kafka/bin/kafka-console-consumer.sh --bootstrap-server localhost:9092  --topic flight_delay_classification_request  --from-beginning

In [125]:
!curl -XPOST 'http://localhost:5000/flights/delays/predict/classify_realtime' -F 'DepDelay=5.0' -F 'Carrier=AA' -F 'FlightDate=2016-12-23' -F 'Dest=ATL' -F 'FlightNum=1519' -F 'Origin=SFO'

{"status": "OK", "id": "b2498c0e-0766-4527-b926-e8bd9f2576b4"}

In [130]:
!curl 'http://localhost:5000/flights/delays/predict/classify_realtime/response/EXAMPLE_UUID_g3t03qtq3t'

{"status": "WAIT", "id": "EXAMPLE_UUID_g3t03qtq3t"}

In [ ]:
db.flight_delay_classification_response.insert(
      {
        id: "EXAMPLE_UUID_g3t03qtq3t",
        prediction: {"test": "data"}
      }
)

In [137]:
!curl 'http://localhost:5000/flights/delays/predict/classify_realtime/response/EXAMPLE_UUID_g3t03qtq3t' | json_pp

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    51  100    51    0     0   2066      0 --:--:-- --:--:-- --:--:-- 51000
{
   "status" : "WAIT",
   "id" : "EXAMPLE_UUID_g3t03qtq3t"
}


In [136]:
from pymongo import MongoClient
client = MongoClient()
prediction = \
        client.agile_data_science.flight_delay_classification_response.find_one(
          {
            "id": "EXAMPLE_UUID_g3t03qtq3t"
} )

In [140]:
response = {"status": "WAIT", "id": "EXAMPLE_UUID_g3t03qtq3t"}
if prediction:
        response["status"] = "OK"
        response["prediction"] = prediction

In [141]:
response

{'id': 'EXAMPLE_UUID_g3t03qtq3t',
 'prediction': {'_id': ObjectId('5a0286d00c19c8c331ed70ee'),
  'id': 'EXAMPLE_UUID_g3t03qtq3t',
  'prediction': {'test': 'data'}},
 'status': 'OK'}

# make_predictions_streaming.py